# Original problem
- You had a base Logger class that would log messages to a file natively.
- If you wanted to log messages to other destinations such as a socket, syslog etc, you had to create separate subclasses for each destination.
- This would lead to a proliferation of subclasses and code duplication.
- Making it harder to maintain the classes. And if you wanted to add new features, you would have to create even more subclasses.

In [14]:
# original Logger class
class Logger:
    def __init__(self, file):
        self.file = file
    def log(self, message):
        self.file.write(message + '\n')
        self.file.flush()


# adding a new output destination (e.g., socket)
class SocketLogger(Logger):
    def __init__(self, socket):
        self.socket = socket
    def log(self, message):
        self.socket.sendall((message + '\n').encode('utf-8'))


# adding another output destination (e.g., database)
class DatabaseLogger(Logger):
    def __init__(self, db_connection):
        self.db_connection = db_connection
    def log(self, message):
        cursor = self.db_connection.cursor()
        cursor.execute("INSERT INTO logs (message) VALUES (?)", (message,))
        self.db_connection.commit()

# Solution 1 ✨ - Adapter Pattern
- Instead of creating many different sub-classes for each logging destination, you can create an Adapter which will make the different logging destinations look like the same file interface the original Logger class expects.
- So, now, you can have the single Logger class and create multiple adapters for different logging destinations.
- You can maintain these adapters in a separate script or module, different from the Logger class so it is easier to maintain, extend etc., in the future.
- The adapter and the Logger objects can be initialized at runtime.


## Duck typing
Adapter pattern relies on duck typing which means that if an object behaves like a certain type (by having the same methods and attributes), it can be treated as that type, regardless of its actual class (and type and class hierarchy are not checked).

In [15]:
# creating adapter for socket class
class SocketAdapter:
    def __init__(self, socket):
        self.socket = socket
    def write(self, message):
        self.socket.sendall((message + '\n').encode('utf-8'))
    def flush(self):
        pass


In [16]:
import sys

logger = Logger(sys.stdout)
logger.log("This is a log message to standard output.")

# creating a socket adapter
# socket_logger = Logger(SocketAdapter(socket))
# socket_logger.log("This is a log message to socket.")

This is a log message to standard output.


# Solution 2 ✨ - Bridge Pattern
- Instead of contorting the output destinations to fit the `file` interface expected by the Logger class, you can separate the abstraction (Logger) from its implementation (output destination).
- In this method, we separate the task of filtering log messages and writing logs to the output destination into two separate class hierarchies; an outer abstraction (filtering) and an inner implementation (writing logs to output destination).
- This pattern presents for more symmetry as the previous Logger class natively supported writing to a file but other output destinations had to be extended via subclasses.
- This avoids the sub-class explosion problem as the 2 class hierarchies can be extended independently, and both objects can be initialized at runtime.

In [17]:
# changing the Logger class to remove the native file output support
class Logger:
    def __init__(self, handler):
        self.handler = handler
    def log(self, message):
        self.handler.emit(message)


# filtering operation is handled in the outer abstraction layer
class FilteredLogger(Logger):
    def __init__(self, pattern, handler):
        self.pattern = pattern
        super().__init__(handler)
    
    def log(self, message):
        if self.pattern in message:
            super().log(message)



# ------ Inner implementation classes (Handlers) --------
class FileHandler:
    def __init__(self, file):
        self.file = file

    def emit(self, message):
        self.file.write(message + '\n')
        self.file.flush()



class SocketHandler:
    def __init__(self, socket):
        self.socket = socket
    
    def emit(self, message):
        self.socket.sendall((message + '\n').encode('ascii'))

    

In [18]:
filehandler = FileHandler(sys.stdout)

logger = Logger(filehandler)

logger.log("Testing bridge pattern...")

Testing bridge pattern...


In [19]:
logger = FilteredLogger('Error', filehandler)

logger.log('Ignored: this will not be logged')
logger.log('Error: this is important')


Error: this is important


# Solution 3 ✨ - Decorator Pattern 
- Bridge pattern contains asymmetry; 
- the method offered by the inner implementation classes, emit(), differs from the one offered by the abstraction classes.
- So more than 1 filters cannot be stacked, as when the other filters are wrapped around the first one, the method offered by the inner implementation is different from the one exposed by the first filter.
- So Decorator pattern fixes this problem by offering the same interface, log(), everywhere.

## i. Problem with Bridge pattern

In [23]:
class Logger:
    def __init__(self, handler):
        self.handler = handler
    def log(self, message):
        self.handler.emit(message)


# filtering operation is handled in the outer abstraction layer
class FilteredLogger(Logger):
    def __init__(self, pattern, handler):
        self.pattern = pattern
        super().__init__(handler)
    
    def log(self, message):
        print(f"Incoming message: {message}\n")
        if self.pattern in message:
            super().log(message)



# ------ Inner implementation classes (Handlers) --------
class FileHandler:
    def __init__(self, file):
        self.file = file

    def emit(self, message):
        self.file.write(message + '\n')
        self.file.flush()



In [24]:
fh = FileHandler(sys.stdout)
f1 = FilteredLogger("Warning", fh)

f2 = FilteredLogger("Warning error", f1)

f1.log("Warning: something something ne")


Incoming message: Warning: something something ne



In [25]:
f2.log("Warning error: something something ne")

Incoming message: Warning error: something something ne



AttributeError: 'FilteredLogger' object has no attribute 'emit'

## ii. Decorator Pattern solution

In [31]:
# now the FilteredLogger class becomes a class itself and retains the same log() method
class Logger:
    def __init__(self, handler):
        self.handler = handler
    def log(self, message):
        self.handler.emit(message)


# filtering operation is handled in the outer abstraction layer
class LogFilter:
    def __init__(self, pattern, logger):
        self.pattern = pattern
        self.logger = logger
    
    def log(self, message):
        if self.pattern in message:
            self.logger.log(message)



# ------ Inner implementation classes (now called __Loggers) also offer the same log() interface --------
class FileLogger:
    def __init__(self, file):
        self.file = file

    def log(self, message):
        self.file.write(message + '\n')
        self.file.flush()



class SocketLogger:
    def __init__(self, socket):
        self.socket = socket
    
    def log(self, message):
        self.socket.sendall((message + '\n').encode('ascii'))

    

In [36]:
fh = FileLogger(sys.stdout)
f1 = LogFilter("Warning", fh)

f2 = LogFilter("Warning error", f1)

print(f"Only 1 filter")
f1.log("Warning: something something ne")

print(f"\n\nStacking 2 filters")
f2.log("Warning error: something something ne")


Only 1 filter


Stacking 2 filters
Warning error: something something ne
